# **1) using GMF (general matrix factorixation)**

In [ ]:
# import all necessary libraries
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Rocket_retail.csv')

In [ ]:
# print the first 10 rows
df.head(10)

,visitorid,ses_rec,ses_rec_avg,ses_rec_sd,ses_rec_cv,user_rec,ses_n,ses_n_r,int_n,int_n_r,...,int_cat16_n,int_cat17_n,int_cat18_n,int_cat19_n,int_cat20_n,int_cat21_n,int_cat22_n,int_cat23_n,int_cat24_n,target_class
0,89,23.0,16.000000,0.000000,0.000000,39.0,2.0,0.051282,3.0,1.500000,...,0,0,0,0,3,0,0,0,0,1
1,251,23.0,61.000000,0.000000,0.000000,85.0,2.0,0.023529,2.0,1.000000,...,0,0,0,0,0,0,1,0,0,1
2,270,14.0,0.100000,0.300000,3.000000,21.0,11.0,0.523810,13.0,1.181818,...,0,0,0,0,0,0,13,0,0,1
3,298,11.0,13.000000,0.000000,0.000000,24.0,2.0,0.083333,4.0,2.000000,...,0,0,0,3,1,0,0,0,0,1
4,474,16.0,26.000000,0.000000,0.000000,43.0,2.0,0.046512,2.0,1.000000,...,2,0,0,0,0,0,0,0,0,1
5,656,21.0,3.000000,0.000000,0.000000,24.0,2.0,0.083333,2.0,1.000000,...,0,0,0,0,0,0,2,0,0,1
6,724,26.0,6.333334,6.944223,1.096456,47.0,4.0,0.085106,13.0,3.250000,...,0,13,0,0,0,0,0,0,0,0
7,1163,16.0,5.000000,0.000000,0.000000,22.0,2.0,0.090909,2.0,1.000000,...,0,0,0,2,0,0,0,0,0,1
8,1288,10.0,4.000000,0.000000,0.000000,14.0,2.0,0.142857,2.0,1.000000,...,0,0,0,2,0,0,0,0,0,1
9,1312,8.0,0.000000,0.000000,-1.000000,8.0,2.0,0.250000,2.0,1.000000,...,0,0,0,0,0,0,2,0,0,1


In [ ]:
print(df.columns)

Index(['visitorid', 'ses_rec', 'ses_rec_avg', 'ses_rec_sd', 'ses_rec_cv',
       'user_rec', 'ses_n', 'ses_n_r', 'int_n', 'int_n_r', 'tran_n',
       'tran_n_r', 'rev_sum', 'rev_sum_r', 'major_spend_r', 'int_cat_n_avg',
       'int_itm_n_avg', 'ses_mo_avg', 'ses_mo_sd', 'ses_ho_avg', 'ses_ho_sd',
       'ses_wknd_r', 'ses_len_avg', 'time_to_int', 'time_to_tran',
       'int_cat1_n', 'int_cat2_n', 'int_cat3_n', 'int_cat4_n', 'int_cat5_n',
       'int_cat6_n', 'int_cat7_n', 'int_cat8_n', 'int_cat9_n', 'int_cat10_n',
       'int_cat11_n', 'int_cat12_n', 'int_cat13_n', 'int_cat15_n',
       'int_cat16_n', 'int_cat17_n', 'int_cat18_n', 'int_cat19_n',
       'int_cat20_n', 'int_cat21_n', 'int_cat22_n', 'int_cat23_n',
       'int_cat24_n', 'target_class'],
      dtype='object')


In [ ]:
print(len(df))

49358


In [ ]:
# Train-test split with 70% for training and 30% for testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42)

# Step 5: Build the GMF model
embedding_dim = 50  # Set the embedding dimension for user embeddings

# Define user input layer
user_input = layers.Input(shape=(1,), name='user')  # user input (visitorid)

# Embedding layer for user (visitorid)
user_embedding = layers.Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim)(user_input)

# Flatten the user embedding
user_vec = layers.Flatten()(user_embedding)

# Input layer for the rest of the features
feature_input = layers.Input(shape=(X_train.shape[1] - 1,), name='features')  # Exclude user ID
features_dense = layers.Dense(64, activation='relu')(feature_input)

# Concatenate the user embeddings and feature dense layers
merged = layers.Concatenate()([user_vec, features_dense])

# Output layer for binary classification (sigmoid activation)
output = layers.Dense(1, activation='sigmoid')(merged)

# Build the model
model = models.Model(inputs=[user_input, feature_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Define class weights to handle class imbalance
class_weights = {0: 1., 1: 50.}  # Example, adjust according to your data

# Add EarlyStopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Step 7: Prepare inputs
X_train_user = X_train[:, :1]  # First column for user ID
X_train_features = X_train[:, 1:]  # Remaining columns for other features

X_test_user = X_test[:, :1]  # First column for user ID
X_test_features = X_test[:, 1:]  # Remaining columns for other features

# Train the model
history = model.fit(
    [X_train_user, X_train_features], y_train,  # visitorid as user and features as behavior/session data
    validation_data=([X_test_user, X_test_features], y_test),
    epochs=10, batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Step 8: Evaluate the model on the test data
y_pred_prob = model.predict([X_test_user, X_test_features])
y_pred = (y_pred_prob > 0.8).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Print the metrics
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"Test Accuracy: {accuracy}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Print train accuracy
train_accuracy = history.history['accuracy'][-1]
print(f"Training Accuracy: {train_accuracy}")


Epoch 1/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8766 - loss: 3.5361 - val_accuracy: 0.8897 - val_loss: 0.6394
Epoch 2/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8865 - loss: 0.7584 - val_accuracy: 0.8897 - val_loss: 0.6274
Epoch 3/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8837 - loss: 0.7746 - val_accuracy: 0.8897 - val_loss: 0.6223
Epoch 4/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8825 - loss: 0.7643 - val_accuracy: 0.8897 - val_loss: 0.6033
Epoch 5/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8850 - loss: 0.7479 - val_accuracy: 0.8897 - val_loss: 0.6092
Epoch 6/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8873 - loss: 0.7276 - val_accuracy: 0.8897 - val_loss: 0.6264
Epoch 7/10
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8827 - loss: 0.7590 - val_accuracy: 0.8899 - val_loss: 0.6122
463/463 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Precision: 0.8899473044183218
F1 Score: 0.9417021

In [ ]:
print(len(X_test)) # samples taken for prediction and testing

14808


In [ ]:
#True Negatives (TN): 9
#The model correctly identified 9 instances where the actual class was negative (0) and predicted negative.
#False Positives (FP): 1624
#The model incorrectly predicted 1 (positive) for 1624 instances where the actual class was 0 (negative).
#False Negatives (FN): 5
#The model incorrectly predicted 0 (negative) for 5 instances where the actual class was 1 (positive).
#True Positives (TP): 13170
#The model correctly identified 13170 instances where the actual class was positive (1) and predicted positive.


#**2) using NeuMF**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, f1_score, accuracy_score, confusion_matrix
import pandas as pd

# Load the data
df = pd.read_csv('/content/Rocket_retail.csv')

# Assuming your dataset has 'visitorid' and other features
X = df.drop(columns=['target_class'])  # drop the target column
y = df['target_class']  # target column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Number of unique visitors (for embedding)
num_users = len(np.unique(X['visitorid']))

# Define embedding dimension for visitorid
embedding_dim = 50  # Latent factors (embedding dimensions)

# Input layers
user_input = layers.Input(shape=(1,), name='visitorid')  # visitorid as input
behavior_input = layers.Input(shape=(X_train.shape[1] - 1,), name='behavior')  # Exclude 'visitorid' from the features

# User embedding
user_embedding = layers.Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
user_vec = layers.Flatten()(user_embedding)

# Concatenate user embedding with other features
x = layers.Concatenate()([user_vec, behavior_input])

# MLP (Multilayer Perceptron) part: Additional Dense layers
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)  # Added another MLP layer
x = layers.Dense(16, activation='relu')(x)  # Added another MLP layer

# Output layer (binary classification, change for regression if needed)
output = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = models.Model(inputs=[user_input, behavior_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare input data for the model
X_train_user = X_train['visitorid'].values  # Extract visitorid for training
X_train_behavior = X_train.drop(columns=['visitorid']).values  # Extract remaining features for training

X_test_user = X_test['visitorid'].values  # Extract visitorid for testing
X_test_behavior = X_test.drop(columns=['visitorid']).values  # Extract remaining features for testing

# Train the model
history = model.fit(
    [X_train_user, X_train_behavior],  # Inputs: visitorid and behavior data
    y_train,  # Target variable
    epochs=10, batch_size=32,
    validation_data=([X_test_user, X_test_behavior], y_test)
)

# Evaluate the model
y_pred_prob = model.predict([X_test_user, X_test_behavior])
y_pred = (y_pred_prob > 0.7).astype(int)

# Calculate precision, F1 score, and accuracy
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Print the metrics
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"Test Accuracy: {accuracy}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

# Print training accuracy (from the history object)
train_accuracy = history.history['accuracy'][-1]  # Get the last epoch's training accuracy
print(f"Training Accuracy: {train_accuracy}")


Epoch 1/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8767 - loss: 50.8233 - val_accuracy: 0.8833 - val_loss: 8.5481
Epoch 2/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8813 - loss: 42.6513 - val_accuracy: 0.8757 - val_loss: 17.3284
Epoch 3/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8822 - loss: 26.6050 - val_accuracy: 0.8784 - val_loss: 2.2204
Epoch 4/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8829 - loss: 4.2693 - val_accuracy: 0.8960 - val_loss: 0.5382
Epoch 5/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8886 - loss: 4.0165 - val_accuracy: 0.8931 - val_loss: 0.3111
Epoch 6/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8904 - loss: 0.3110 - val_accuracy: 0.8940 - val_loss: 0.3094
Epoch 7/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8910 - loss: 0.3087 - val_accuracy: 0.8959 - val_loss: 0.3059
Epoch 8/10
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8899 - loss: 0.373

In [ ]:
# Here’s an explanation of each value based on the confusion matrix results
# True Negatives (TN = 240): The model correctly predicted negative (class 0) when the actual class was negative.
# False Positives (FP = 839): The model incorrectly predicted positive (class 1) when the actual class was negative.
# False Negatives (FN = 314): The model incorrectly predicted negative (class 0) when the actual class was positive.
# True Positives (TP = 8479): The model correctly predicted positive (class 1) when the actual class was positive.